In [1]:
import os
import torch
import pandas as pd
import scanpy as sc

In [2]:
import SpatialGlue

### Import packages and read data

In [3]:
from numpy.random import default_rng
import scanpy as sc
# import squidpy as sq
from anndata import AnnData
import scipy
# sc.logging.print_header()
from sklearn.metrics.cluster import adjusted_rand_score
import numpy as np
import pandas as pd
import seaborn as sns
import os
import torch
import pandas as pd
import scanpy as sc
from sklearn import metrics
import multiprocessing as mp
from sklearn.metrics.cluster import normalized_mutual_info_score
from numpy import genfromtxt

In [4]:
# Read data
# data_name = 'CBMCs_nCells_1002_nGenes_100_'
data_name = 'CBMCs_nCells_1992_nGenes_100_'
# data_name = 'CBMCs_nCells_1992_nGenes_500_'
# data_name = 'CBMCs_nCells_1992_nGenes_1000_'
# data_name = 'CBMCs_nCells_4970_nGenes_100_'
# data_name = 'CBMCs_nCells_4970_nGenes_500_'
# data_name = 'CBMCs_nCells_9471_nGenes_100_'
# data_name = 'CBMCs_nCells_10201_nGenes_100_'
RNA_counts = pd.read_csv('/data/hoan/spatial_transcriptomics/GraphBGM/Data/scDesign3/'+data_name +'RNA.csv', delimiter=',')
ADT_counts = pd.read_csv('/data/hoan/spatial_transcriptomics/GraphBGM/Data/scDesign3/'+data_name +'ADT.csv', delimiter=',')
metadata =pd.read_csv('/data/hoan/spatial_transcriptomics/GraphBGM/Data/scDesign3/'+data_name +'metadata.csv', delimiter=',') 
# Duplicate
RNA_counts = pd.concat([RNA_counts.copy().add_suffix(f'_{i+1}') for i in range(10)], axis=1)
# ADT_counts = pd.concat([ADT_counts.copy().add_suffix(f'_{i+1}') for i in range(10)], axis=1)

In [5]:
RNA_counts.shape, ADT_counts.shape

((1992, 1000), (1992, 6))

In [6]:
# Create anndata obj
adata_omics1 = AnnData(RNA_counts.values, obsm={"spatial": metadata.iloc[:,2:4].values}, uns={"spatial": metadata.iloc[:,2:4].values})
adata_omics1.obs_names = [f"Cell_{i:d}" for i in range(adata_omics1.n_obs)]
adata_omics1.var_names = list(RNA_counts.columns.values)
adata_omics2 = AnnData(ADT_counts.values, obsm={"spatial": metadata.iloc[:,2:4].values}, uns={"spatial": metadata.iloc[:,2:4].values})
adata_omics2.obs_names = [f"Cell_{i:d}" for i in range(adata_omics1.n_obs)]
adata_omics2.var_names = list(ADT_counts.columns.values)

In [7]:
adata_omics1, adata_omics2

(AnnData object with n_obs × n_vars = 1992 × 1000
     uns: 'spatial'
     obsm: 'spatial',
 AnnData object with n_obs × n_vars = 1992 × 6
     uns: 'spatial'
     obsm: 'spatial')

In [8]:
# # read data
# file_fold = '/data/hoan/spatial_transcriptomics/GraphBGM/Data/SpatialGlue' #please replace 'file_fold' with the download path
# adata_omics1 = sc.read_h5ad(file_fold + '/adata_RNA.h5ad')
# adata_omics2 = sc.read_h5ad(file_fold + '/adata_ADT.h5ad')
# adata_omics1.var_names_make_unique()
# adata_omics2.var_names_make_unique()

### Preprocessing data (SpatialGlue)

In [9]:
# Specify data type
data_type = '10x'
# Fix random seed
from SpatialGlue.preprocess import fix_seed
random_seed = 2022
fix_seed(random_seed)

In [10]:
from SpatialGlue.preprocess import clr_normalize_each_cell, pca

In [11]:
sc.pp.filter_genes(adata_omics2, min_cells=0)
adata_omics2 = adata_omics2[adata_omics1.obs_names].copy()

In [12]:
# n_PCAs = adata_omics2.n_vars-1
n_PCAs = 5

In [13]:
# Protein
adata_omics2 = clr_normalize_each_cell(adata_omics2)
adata_omics2.obsm['feat'] = pca(adata_omics2, n_comps=n_PCAs)

In [14]:
from scipy.sparse import issparse, csr_matrix
# Kiểm tra dữ liệu và chuyển đổi nếu cần
if not issparse(adata_omics2.X):
    adata_omics2.X = csr_matrix(adata_omics2.X)

In [15]:
#RNA
sc.pp.filter_genes(adata_omics1, min_cells=0)
sc.pp.filter_cells(adata_omics1, min_genes=0)

In [16]:
# sc.pp.highly_variable_genes(adata_omics1, flavor="seurat_v3", n_top_genes=500)
sc.pp.normalize_total(adata_omics1, target_sum=1e4)
sc.pp.log1p(adata_omics1)

In [17]:
# adata_omics1_high =  adata_omics1[:, adata_omics1.var['highly_variable']]
# adata_omics1.obsm['feat'] = pca(adata_omics1_high, n_comps=adata_omics2.n_vars-1)
adata_omics1.obsm['feat'] = pca(adata_omics1, n_comps=n_PCAs)

In [18]:
# adata_omics1.obsm['feat']

In [19]:
import pandas as pd

df_omics1 = pd.DataFrame(adata_omics1.X, index=adata_omics1.obs_names)
df_omics2 = pd.DataFrame(adata_omics2.X, index=adata_omics2.obs_names)
common_samples = df_omics1.index.intersection(df_omics2.index)
adata_omics2 = adata_omics2[adata_omics2.obs.index.isin(common_samples)]
adata_omics1 = adata_omics1[adata_omics1.obs.index.isin(common_samples)]

In [20]:
true_labels = np.array(metadata['label'])
n_clusters = len(set(true_labels))
n_clusters

15

### Run SpatialGlue

In [21]:
if True:
    from SpatialGlue.preprocess import construct_neighbor_graph
    data = construct_neighbor_graph(adata_omics1, adata_omics2, datatype=data_type)
    from SpatialGlue.SpatialGlue_pyG import Train_SpatialGlue
    import torch
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    print(f"Using device: {device}")
    model = Train_SpatialGlue(data, datatype=data_type, device=device)
    # train model
    output = model.train()
    adata = adata_omics1.copy()
    adata.obsm['emb_latent_omics1'] = output['emb_latent_omics1']
    adata.obsm['emb_latent_omics2'] = output['emb_latent_omics2']
    adata.obsm['SpatialGlue'] = output['SpatialGlue']
    adata.obsm['alpha'] = output['alpha']
    adata.obsm['alpha_omics1'] = output['alpha_omics1']
    adata.obsm['alpha_omics2'] = output['alpha_omics2']
    # clustering
    # we set 'mclust' as clustering tool by default. Users can also select 'leiden' and 'louvain'
    from SpatialGlue.utils import clustering
    os.environ['R_HOME'] = '/data/hoan/mybin/miniconda3/lib/R'
    tool = 'mclust' # mclust, leiden, and louvain
    clustering(adata, key='SpatialGlue', add_key='SpatialGlue', n_clusters=n_clusters, method=tool, use_pca=True, start=0.9, end=1.1, increment=0.02)
    print(adjusted_rand_score(true_labels, adata.obs['SpatialGlue']))
    print(normalized_mutual_info_score(true_labels, adata.obs['SpatialGlue']))

/home/vanhoan310/miniconda3/envs/gnn/lib/python3.8/site-packages/SpatialGlue/preprocess.py:41: ImplicitModificationWarning: Trying to modify attribute `._uns` of view, initializing view as actual.
  adata_omics1.uns['adj_spatial'] = adj_omics1
/home/vanhoan310/miniconda3/envs/gnn/lib/python3.8/site-packages/SpatialGlue/preprocess.py:46: ImplicitModificationWarning: Trying to modify attribute `._uns` of view, initializing view as actual.
  adata_omics2.uns['adj_spatial'] = adj_omics2


Using device: cpu


/home/vanhoan310/miniconda3/envs/gnn/lib/python3.8/site-packages/SpatialGlue/preprocess.py:132: UserWarning: torch.sparse.SparseTensor(indices, values, shape, *, device=) is deprecated.  Please use torch.sparse_coo_tensor(indices, values, shape, dtype=, device=). (Triggered internally at ../torch/csrc/utils/tensor_new.cpp:641.)
  return torch.sparse.FloatTensor(indices, values, shape)
  0%|          | 0/200 [00:00<?, ?it/s]/home/vanhoan310/miniconda3/envs/gnn/lib/python3.8/site-packages/SpatialGlue/model.py:212: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  self.alpha = F.softmax(torch.squeeze(self.vu) + 1e-6)
100%|██████████| 200/200 [07:04<00:00,  2.12s/it]


Model training finished!



During startup - Warning messages:
1: package ‘methods’ was built under R version 3.6.1 
2: package ‘datasets’ was built under R version 3.6.1 
3: package ‘utils’ was built under R version 3.6.1 
4: package ‘grDevices’ was built under R version 3.6.1 
5: package ‘graphics’ was built under R version 3.6.1 
6: package ‘stats’ was built under R version 3.6.1 
R[write to console]:     __  ___________    __  _____________
   /  |/  / ____/ /   / / / / ___/_  __/
  / /|_/ / /   / /   / / / /\__ \ / /   
 / /  / / /___/ /___/ /_/ /___/ // /    
/_/  /_/\____/_____/\____//____//_/    version 5.4.7
Type 'citation("mclust")' for citing this R package in publications.



fitting ...
  |======================================================================| 100%
0.2911218605609437
0.5290312782908388


In [22]:
# # visualization
# import matplotlib.pyplot as plt
# adata.obsm['spatial'][:,1] = -1*adata.obsm['spatial'][:,1]

# fig, ax_list = plt.subplots(2, 2, figsize=(10, 6))
# # sc.pp.neighbors(adata, use_rep='SpatialGlue', n_neighbors=30)
# # sc.tl.umap(adata)
# adata.obs['domain'] = true_labels.astype(str)
# sc.pl.umap(adata, color='SpatialGlue', ax=ax_list[0,0], title='SpatialGlue', s=20, show=False)
# sc.pl.embedding(adata, basis='spatial', color='SpatialGlue', ax=ax_list[0,1], title='SpatialGlue', s=20, show=False)
# sc.pl.umap(adata, color='domain', ax=ax_list[1,0], title='domain', s=20, show=False)
# sc.pl.embedding(adata, basis='spatial', color='domain', ax=ax_list[1,1], title='domain', s=20, show=False)

# plt.tight_layout(w_pad=0.3)
# plt.show()

In [23]:
# # plotting with annotation
# fig, ax_list = plt.subplots(1, 2, figsize=(9.5, 3))
# sc.pp.neighbors(adata, use_rep='SpatialGlue', n_neighbors=30)
# sc.tl.umap(adata)

# sc.pl.umap(adata, color='SpatialGlue', ax=ax_list[0], title='SpatialGlue', s=10, show=False)
# sc.pl.embedding(adata, basis='spatial', color='SpatialGlue', ax=ax_list[1], title='SpatialGlue', s=20, show=False)

# ax_list[0].get_legend().remove()

# plt.tight_layout(w_pad=0.3)
# plt.show()

### GraphGBM

In [24]:
from sklearn.decomposition import PCA
# from GraphST.utils import refine_label
from sklearn.preprocessing import StandardScaler
# from GraphST.utils import mclust_R
import numpy as np
from numpy import dot, array
from sklearn.cross_decomposition import CCA
from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import kneighbors_graph
from sklearn.mixture import BayesianGaussianMixture
from mvlearn.datasets import load_UCImultifeature
from mvlearn.embed import GCCA
# gcca = GCCA(n_components = 20)
# Xs_latents = gcca.fit_transform(Xs)

from mvlearn.plotting import crossviews_plot
from mvlearn.embed import DCCA
from mvlearn.embed import MCCA
from mvlearn.embed import KMCCA

In [25]:
# embedding_RNA.shape

In [26]:
embedding = adata_omics1.obsm['feat']
n_neighbors = 4
connectivity = kneighbors_graph(adata_omics1.obsm['spatial'], n_neighbors=n_neighbors, include_self=False)
# make connectivity symmetric
connectivity = 0.5 * (connectivity + connectivity.T)
embedding_RNA = connectivity.dot(embedding)
adata_omics1.obsm['spatial_RNAfeat'] = embedding_RNA

embedding = adata_omics2.obsm['feat']
# embedding = adata_omics1.obsm['feat']
# connectivity = kneighbors_graph(adata_omics1.obsm['spatial'], n_neighbors=n_neighbors, include_self=False)
# # make connectivity symmetric
# connectivity = 0.5 * (connectivity + connectivity.T)
embedding_Pro = connectivity.dot(embedding)
adata_omics1.obsm['spatial_Profeat'] = embedding_Pro

# pca = PCA(n_components=10, random_state=42) 
# RNA_pca_embedding = pca.fit_transform(embedding_RNA)

# pca = PCA(n_components=5, random_state=42) 
# Pro_pca_embedding = pca.fit_transform(embedding_Pro)

# Standardize the data
scaler_a = StandardScaler()
scaler_b = StandardScaler()

data_a_train = scaler_a.fit_transform(embedding_RNA)
data_b_train = scaler_b.fit_transform(embedding_Pro)

# Define and train the CCA model
n_components = 5  # Number of canonical components
cca = CCA(n_components=n_components)
cca.fit(data_a_train, data_b_train)

#adata.obsm['emb_pca'] = np.concatenate((embedding_RNA/np.linalg.norm(embedding_RNA), embedding_Pro), axis=1)

CCA(n_components=5)

In [27]:
data_a_train.shape, data_b_train.shape

((1992, 5), (1992, 5))

In [28]:
### SOTA
from sklearn.mixture import BayesianGaussianMixture
from mvlearn.datasets import load_UCImultifeature
from mvlearn.embed import GCCA
# gcca = GCCA(n_components = 20)
# Xs_latents = gcca.fit_transform(Xs)

from mvlearn.plotting import crossviews_plot
from mvlearn.embed import DCCA
from mvlearn.embed import MCCA
from mvlearn.embed import KMCCA
true_labels = np.array(metadata['label'])
n_clusters = len(set(true_labels))
# Xs = [data_a_train, data_b_train] # multiview data
Xs = [data_a_train-data_a_train.min(), data_b_train-data_b_train.min()] # multiview data
# Xs = [embedding_RNA-embedding_RNA.min(), embedding_Pro-embedding_Pro.min()] # multiview data
# Xs = [RNA_pca_embedding-RNA_pca_embedding.min(), Pro_pca_embedding-Pro_pca_embedding.min()] # multiview data
# mcca = MCCA(n_components = 20, regs = 1)
mcca = KMCCA(n_components = 20, kernel = 'poly', regs = 0)
mcca.fit(Xs)
# MCCA()
Xs_latents = mcca.transform(Xs)
adata_omics1.obsm['emb_pca'] = np.concatenate((Xs_latents[0,:,:], Xs_latents[1,:,:]), axis=1)
# adata_omics1.obsm['emb_pca'] = np.concatenate((embedding_RNA, embedding_Pro), axis=1)

In [29]:
gmm = BayesianGaussianMixture(n_components=n_clusters, covariance_type='full', random_state=0, init_params = 'random_from_data', n_init = 5, max_iter = 1000)
# gmm.means_init = np.array([embedding[initial_labels == i].mean(axis=0) for i in range(n_clusters)])
# Step 4: Fit GMM
gmm.fit(adata_omics1.obsm['emb_pca'])
cluster_labels = gmm.predict(adata_omics1.obsm['emb_pca'])
adata_omics1.obs['domain'] = cluster_labels  
from utils import cKD_refine_label
refine_cluster_labels = cKD_refine_label(np.array(adata_omics1.obsm['spatial']), cluster_labels, k = 45)
print('GBM - no post-preprocessing')
print(adjusted_rand_score(true_labels[true_labels>=0], cluster_labels[true_labels>=0]))
print(normalized_mutual_info_score(true_labels[true_labels>=0], cluster_labels[true_labels>=0]))
print('GBM - with post-preprocessing')
print(adjusted_rand_score(true_labels[true_labels>=0], refine_cluster_labels[true_labels>=0]))
print(normalized_mutual_info_score(true_labels[true_labels>=0], refine_cluster_labels[true_labels>=0]))

GBM - no post-preprocessing
0.26221360549930056
0.47642927050469097
GBM - with post-preprocessing
0.28531060260505087
0.5373047164460094


In [30]:
for key in ['spatial_RNAfeat', 'spatial_Profeat', 'emb_pca']:
    if 'feat' in key:
        use_pca = False
    else:
        use_pca = True
    tool = 'mclust' # mclust, leiden, and louvain
    # clustering(adata_omics1, key='emb_pca', add_key='GBM', n_clusters=n_clusters, method=tool, use_pca=False, increment=0.1)
    clustering(adata_omics1, key=key, add_key='GBM', n_clusters=n_clusters, method=tool, use_pca=use_pca, increment=0.1)
    cluster_labels = np.array(adata_omics1.obs['GBM'])
    from utils import cKD_refine_label
    refine_cluster_labels = cKD_refine_label(np.array(adata_omics1.obsm['spatial']), cluster_labels, k = 45)
    print('------------------------------------------  ', key, '  ------------------------------------------')
    print('Data:', data_name)
    print('GBM - no post-preprocessing')
    print(adjusted_rand_score(true_labels[true_labels>=0], cluster_labels[true_labels>=0]))
    print(normalized_mutual_info_score(true_labels[true_labels>=0], cluster_labels[true_labels>=0]))
    print('GBM - with post-preprocessing')
    print(adjusted_rand_score(true_labels[true_labels>=0], refine_cluster_labels[true_labels>=0]))
    print(normalized_mutual_info_score(true_labels[true_labels>=0], refine_cluster_labels[true_labels>=0]))
    print('SpatialGlue')
    print(adjusted_rand_score(true_labels[true_labels>=0], adata.obs['SpatialGlue'][true_labels>=0]))
    print(normalized_mutual_info_score(true_labels[true_labels>=0], adata.obs['SpatialGlue'][true_labels>=0]))

fitting ...
  |======================================================================| 100%
------------------------------------------   spatial_RNAfeat   ------------------------------------------
Data: CBMCs_nCells_1992_nGenes_100_
GBM - no post-preprocessing
0.2705134614819952
0.49559176792281556
GBM - with post-preprocessing
0.354738795689078
0.5880647331981541
SpatialGlue
0.2911218605609437
0.5290312782908388
fitting ...
  |======================================================================| 100%
------------------------------------------   spatial_Profeat   ------------------------------------------
Data: CBMCs_nCells_1992_nGenes_100_
GBM - no post-preprocessing
0.2366485023046793
0.4607413195429489
GBM - with post-preprocessing
0.2741829107084423
0.5561126582683208
SpatialGlue
0.2911218605609437
0.5290312782908388
fitting ...
  |======================================================================| 100%
------------------------------------------   emb_pca   -----------------